## Build Training set for doc types

In [23]:
import TWB
import glob

In [24]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sparaschiakos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
english_document_data = pd.read_pickle('data/english_document_data.pkl')
stems = pd.read_pickle('data/stemmed_data.pkl')
freq_content = pd.read_pickle('data/freq_content.pkl')

In [4]:
# defining function to check buzz word 
def exist_(temp,j): 
    temp2=temp.lower()
    temp3=temp2.find(j)
    return temp3>0
    
dictionary2=['certificate','instruction','poster','protocol', 'presentation',
             'response','letter','report','map','manual','banner']

for j in dictionary2:
    jj=english_document_data.Filename.apply(lambda x: exist_(x,j))
    rr=pd.DataFrame(columns=[str(j)])
    rr[str(j)]=jj
    english_document_data=pd.concat([english_document_data,rr ], axis=1)

In [5]:
# keep only the english ones
stems_en = stems.iloc[english_document_data.index]
freq_content_en = freq_content.iloc[english_document_data.index]

In [6]:
# drop dublicates and create the corpus
stems_en_noDbl = stems_en.drop_duplicates(subset='Content')
len(stems_en_noDbl.index)

6728

In [7]:
# create corpus
corpus = ''
for text in stems_en_noDbl.Content:
    corpus = corpus + text
# tokenize corpus
corpus_tok = tokenizer.tokenize(corpus)

In [8]:
corpus_tok = list(set(corpus_tok))
print(len(corpus_tok))

117229


In [9]:
# drop digits
corpus_tok_noDgt = []
for word in corpus_tok:
    if not word.isdigit():
        corpus_tok_noDgt.append(word)
len(corpus_tok_noDgt)

107119

In [10]:
# keep df with true in labels
df = english_document_data.loc[
    (english_document_data['certificate']==True) |
    (english_document_data['instruction']==True) |
    (english_document_data['poster']==True) |
    (english_document_data['protocol']==True) |
    (english_document_data['presentation']==True) |
    (english_document_data['response']==True) |
    (english_document_data['letter']==True) |
    (english_document_data['report']==True) |
    (english_document_data['map']==True) |
    (english_document_data['manual']==True) |
    (english_document_data['banner']==True)]

In [11]:
labeled_docs = pd.DataFrame(index= df.index, columns=['document', 'label'])

In [12]:
for i in df.index:
    go = False
    for j in df.columns:
        if (df[j][i] == True & go == False):
            labeled_docs.document[i] = df.Filename[i]
            labeled_docs.label[i] = j
            go = True

In [ ]:
texts =[]

In [ ]:
texts = list(stems_en_noDbl.Content)

In [ ]:
token_freq = [ TWB.common.freq(t.split(' ')) for t in texts ]

In [ ]:
token_freq[0]

In [ ]:
dictionary = list(set(' '.join(texts).split(' ')))
dict_idx = { w : i for (i,w) in enumerate(dictionary) }
dict_idx

In [ ]:
import numpy as np
M = np.zeros((len(texts),len(dictionary)))

In [ ]:
for i, text in enumerate(texts):
    M[i,[dict_idx[w] for w in token_freq[i].keys()]] = [x/sum(token_freq[i].values()) for x in  list(token_freq[i].values())]
    print(list(token_freq[i].values()))
M

In [ ]:
labeled_set = pd.DataFrame(M, columns=dictionary)

In [ ]:
labeled_set.head()

In [ ]:
digit_stems = []
for col in labeled_set.columns:
    if col.isdigit():
        digit_stems.append(col)        

In [ ]:
len(digit_stems)

In [ ]:
labeled_set['digits'] = labeled_set[digit_stems].apply(sum,axis=1)

In [ ]:
labeled_set.hea

In [ ]:
labeled_set = labeled_set.drop(digit_stems, axis=1)

In [ ]:
len(labeled_set.columns)

In [ ]:
labeled_set.to_pickle('data/labeled_set_type.pkl')

In [13]:
stems = pd.read_pickle('data/labeled_set_type.pkl')

In [14]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(stems)

# Apply transform to both the training set and the test set.
train_img = scaler.transform(stems)

In [16]:
from sklearn.decomposition import PCA
pca = PCA(.95)

In [ ]:
pca.fit(train_img)

In [ ]:
from pandas.tools.plotting import parallel_coordinates
import pandas as pd
from sklearn import svm

from sklearn.metrics import jaccard_similarity_score

#from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

random_state = 100
#import xgboost as xgb

import pickle
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from sklearn.model_selection import cross_val_score

In [ ]:
pca.n_components_

In [26]:
from sklearn.model_selection import train_test_split

#X_train, X_test, Y_train, Y_test = train_test_split(train_img[labeled_docs.index], labeled_docs['label'], test_size=0.3)
#print(X_train.shape, Y_train.shape)
#print(X_test.shape, Y_test.shape)
#stems.loc[labeled_docs.index]

english_document_data
#english_document_data.loc[labeled_docs.index]
#stems
#labeled_docs.index


,Date,Wordcount,Filename,Source_lang,Source_country,Target_lang,Target_country,NGO,Format,System,Content,Average Sentence Length
0,2016-12-12 00:56:11,109.0,347words.docx,en,US,fr,FR,British Red Cross,doc,Kató 1,This is a test. This is a test. This is a test...,5.046512
1,2016-12-12 09:30:28,109.0,347words.docx,en,US,fr,FR,British Red Cross,doc,Kató 1,This is a test. This is a test. This is a test...,5.046512
2,2016-12-19 11:04:31,0.0,347words.docx,en,US,fr,FR,British Red Cross,doc,Kató 1,This is a test. This is a test. This is a test...,5.046512
3,2016-12-19 12:09:22,109.0,347words.docx,en,US,fr,FR,Translators without Borders,doc,Kató 1,This is a test. This is a test. This is a test...,5.046512
4,2016-12-19 12:10:10,109.0,347words.docx,en,US,fr,FR,British Red Cross,doc,Kató 1,This is a test. This is a test. This is a test...,5.046512
5,2017-03-28 11:42:43,109.0,347words.docx,en,US,fr,FR,Translators without Borders,doc,Kató 1,This is a test. This is a test. This is a test...,5.046512
6,2017-06-15 16:33:45,109.0,347words.docx,en,GB,ar,SA,Translators without Borders,doc,Kató 1,This is a test. This is a test. This is a test...,5.046512
7,2016-12-12 01:53:23,121.0,387words.docx,en,US,fr,FR,Translators without Borders,doc,Kató 1,This is a test. This is a test. This is a test...,5.041667
8,2016-12-12 12:57:12,121.0,387words.docx,en,US,fr,FR,British Red Cross,doc,Kató 1,This is a test. This is a test. This is a test...,5.041667
9,2017-02-22 10:52:02,121.0,387words.docx,en,GB,ky,KG,Translators without Borders,doc,Kató 1,This is a test. This is a test. This is a test...,5.041667


In [22]:
stems_en_noDbl.loc[labeled_docs['document'].drop_duplicates().index]

,Filename,Content
205,english volunt newslett januari 2017 docx,updat educ program team import news oper smile...
461,201702 letter ngo committe migrat docx,letter ngo committe migrat the ngo committe mi...
692,label emerg respons map docx,1 creat 2 data sourc 3 map document 4 project ...
1064,CP protocol final lstmh approv docx,survey plasmodium falciparum sulfadoxin pyrime...
1191,2a data inform epidem respons idsr pptx,data inform epidem control idsr Mr christian E...
1193,2b laboratori network support outbreak prepare...,laboratori network support outbreak surveil re...
1195,publish report 2016 final docx,publish report 2016 cancer intellig tabl conte...
1202,cafe newslett apr 2017 translat 1 docx,cafe news april 2017 017 welcom cafe welcom ap...
1206,cafe newslett apr 2017 translat 2 docx,dutch legend ruud van nistelrooy discuss impor...
1419,bronz annual report templat docx,school busi logo annual report An updat busi y...
